In [14]:
import numpy as np 
import cv2 as cv
from math import atan2, degrees

scan_source = cv.VideoCapture(0)
bol,frame = scan_source.read()
live_cam_shape = frame.shape
live_cam_shape = (live_cam_shape[1],live_cam_shape[0])

target_image = cv.imread("cat3.jpg",cv.COLOR_BGR2RGB)
shape1 = target_image.shape
print(shape1)

augment = cv.VideoCapture("cat1.mp4")
bol1,augment_image1 = augment.read()

# initializing shape 
resizeed1 = target_image.shape
print(resizeed1)
ideal_shape = (resizeed1[1],resizeed1[0])


# angle calculation to avoid glitch image
def angle_between(scan_p):
    p_1 = scan_p[0]
    p_2 = scan_p[1]
    p_3 = scan_p[2]

    point1 = p_1[0]
    point2 = p_2[0]
    point3 = p_3[0]  

    x1, y1 = point1
    x2, y2 = point2
    x3, y3 = point3
    deg1 = (360 + degrees(atan2(x1 - x2, y1 - y2))) % 360
    deg2 = (360 + degrees(atan2(x3 - x2, y3 - y2))) % 360
    return deg2 - deg1 if deg1 <= deg2 else 360 - (deg1 - deg2)

# orb 
orb = cv.ORB.create(10000)

# Brute-Force Matching with ORB
bf = cv.BFMatcher(cv.NORM_HAMMING, crossCheck=True)

# target image kp and des
gray1 = cv.cvtColor(target_image,cv.COLOR_BGR2GRAY)
kp1,des1 = orb.detectAndCompute(gray1,None)

while True:
    bol1,augment_image1 = augment.read()
    bol,frame = scan_source.read()
    if bol == False:
        break
    if bol1 == False:
        augment.set(cv.CAP_PROP_POS_FRAMES, 0)
        continue
    augment_image1 = cv.resize(augment_image1,ideal_shape)

    # live frame kp and des
    gray2 = cv.cvtColor(frame,cv.COLOR_BGR2GRAY)
    kp2,des2 = orb.detectAndCompute(gray2,None)

    matches = bf.match(des1,des2)

    # Sort them in the order of their distance.
    matches = sorted(matches, key = lambda x:x.distance)

    # minimum matches
    if len(matches) > 10:

        src_pts = np.float32([ kp1[i.queryIdx].pt for i in matches[:50]])
        dst_pts = np.float32([ kp2[i.trainIdx].pt for i in matches[:50]])

        # find homography 
        matrx,mask = cv.findHomography(src_pts,dst_pts,cv.RANSAC,5.0)

        target_points = np.float32([ [0,0],[0,shape1[0]] , [shape1[1],shape1[0]] ,[shape1[1],0]] ).reshape(-1,1,2)
        
        scan_points =  cv.perspectiveTransform(target_points,matrx)

        ang = int(angle_between(scan_points))

        # glitch filtering
        if ang in range(240,290):
            
            aug_frame = cv.warpPerspective(augment_image1,matrx,live_cam_shape)

            mask_frame = cv.fillConvexPoly(frame,scan_points.astype(int),(0,0,0))

           

            final_frame = aug_frame+mask_frame

            cv.imshow("f",final_frame)
        
        else:
            cv.imshow("f",frame)

    if cv.waitKey(1) == ord("q"):
        break
  
scan_source.release()
augment.release()
cv.destroyAllWindows()

(198, 228, 3)
(198, 228, 3)


In [5]:
import cv2 as cv

augment = cv.VideoCapture("cat1.mp4")
bol1,augment_image1 = augment.read()

while True:
    bol1,augment_image1 = augment.read()
    if bol1 == False:
        augment.set(cv.CAP_PROP_POS_FRAMES, 0)
        continue
    cv.imshow("gif",augment_image1)
    if cv.waitKey(10) == ord("q"):
        break
augment.release()
cv.destroyAllWindows()
